# Weight of Evidence (WOE) and Standard Errors

Author: https://www.github.com/deburky

This notebook demonstrates the relationship between Weight of Evidence (WOE), log odds, and their standard errors. We will show that:

1. **WOE is a centered version of log odds** - subtracting the prior log odds
2. **WOE and log odds have identical standard errors** - because subtracting a constant doesn't affect variance
3. **Standard errors can be calculated from contingency tables** and match logistic regression results

## Theoretical Background

### Key Definitions:
- **Log odds for group**: $\theta_1 = \log\left(\frac{n_{11}}{n_{10}}\right)$
- **Prior log odds**: $\theta_{\text{prior}} = \log\left(\frac{n_{\text{pos}}}{n_{\text{neg}}}\right)$  
- **Weight of Evidence**: $\text{WOE}_1 = \theta_1 - \theta_{\text{prior}}$

### Standard Error Properties:
- **SE of log odds**: $\text{SE}(\theta_1) = \sqrt{\frac{1}{n_{11}} + \frac{1}{n_{10}}}$
- **SE of WOE**: $\text{SE}(\text{WOE}_1) = \text{SE}(\theta_1)$ (constant subtraction doesn't change variance)

## 1. Sample Data Setup

We'll work with a simple 2x2 contingency table to demonstrate the concepts:

| Color | Good (y=0) | Bad (y=1) | Total |
|-------|------------|-----------|-------|
| Red   | 10         | 30        | 40    |
| Blue  | 20         | 10        | 30    |
| Total | 30         | 40        | 70    |

From this table we can calculate:
- **Event rates**: Red = 30/40 = 0.75, Blue = 10/30 = 0.333
- **Prior rate**: 40/70 = 0.571
- **Log odds**: Red = ln(30/10) = 1.099, Blue = ln(10/20) = -0.693


In [54]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from fisher_scoring import LogisticRegression
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
from scipy.special import logit

from fastwoe import FastWoe

# Create a rich Console for Jupyter notebook
console = Console()

In [ ]:
# Create the contingency table
data = {"Color": ["Red", "Blue"], "Good": [10, 20], "Bad": [30, 10]}
df = pd.DataFrame(data)

print("Contingency Table:")
table = Table(title="Contingency Table")
table.add_column("Color", justify="center")
table.add_column("Good", justify="center")
table.add_column("Bad", justify="center")
table.add_column("Total", justify="center")

table.add_row("Red", "10", "30", "40")
table.add_row("Blue", "20", "10", "30")
table.add_row("Total", "30", "40", "70")
console.print(table)

# Convert to individual observations (Bernoulli format)
rows = []
for _, row in df.iterrows():
    # Add 'Bad' observations (Target=1)
    rows += [{"Color": row["Color"], "Target": 1}] * row["Bad"]
    # Add 'Good' observations (Target=0)
    rows += [{"Color": row["Color"], "Target": 0}] * row["Good"]
full_df = pd.DataFrame(rows)

# Binary encode Color: Red=0, Blue=1
full_df["Color_bin"] = (full_df["Color"] == "Blue").astype(int)

print(f"\nConverted to {len(full_df)} individual observations")

## 2. Maximum Likelihood Logistic Regression

Let's fit a logistic regression model and examine the standard errors. The model will be:
$$\text{logit}(P(\text{Target}=1)) = \beta_0 + \beta_1 \cdot \text{Color\_bin}$$

Where Color_bin = 0 for Red and 1 for Blue.


In [ ]:
# Fit logistic regression using statsmodels
X = sm.add_constant(full_df["Color_bin"])
y = full_df["Target"]
model = sm.Logit(y, X)
result = model.fit(disp=False)

print("Statsmodels Logistic Regression Results:")
print(f"Intercept (β₀): {result.params['const']:.4f} (SE: {result.bse['const']:.4f})")
print(
    f"Color_bin (β₁): {result.params['Color_bin']:.4f} (SE: {result.bse['Color_bin']:.4f})"
)

# Fit using Fisher Scoring implementation
x = full_df[["Color_bin"]]
logistic = LogisticRegression()
logistic.fit(x, y)
logistic.display_summary(style="cyan1")


### Interpretation of Results:

1. **Intercept (1.0986)**: This is the log odds for the reference group (Red, Color_bin=0)
   - Log odds = ln(30/10) = ln(3) ≈ 1.099 ✓

2. **Coefficient (-1.7918)**: This is the difference in log odds between Blue and Red
   - Blue log odds: ln(10/20) = ln(0.5) ≈ -0.693
   - Red log odds: ln(30/10) = ln(3) ≈ 1.099  
   - Difference: -0.693 - 1.099 = -1.792 ≈ -1.7918 ✓

3. **Standard Errors**: These match the theoretical formulas from contingency table analysis


## 3. Weight of Evidence (WOE) - Inference with Likelihood Ratios

Now let's calculate WOE values and examine their standard errors. WOE transforms each group's log odds by subtracting the overall prior log odds:

$$
WOE = \log \left( \frac{P(y=1|x)}{P(y=0|x)} \right) - \log \left( \frac{P(y=1)}{P(y=0)} \right)
$$

In [ ]:
# Fit WOE encoder
fastwoe_encoder = FastWoe()
fastwoe_encoder.fit(x, y)

# Display feature statistics
print("WOE Feature Statistics:")
feature_stats = fastwoe_encoder.get_feature_stats()
display(feature_stats)

print(f"\nOverall prior probability: {fastwoe_encoder.y_prior_:.4f}")
print(
    f"Prior log odds: {np.log(fastwoe_encoder.y_prior_ / (1 - fastwoe_encoder.y_prior_)):.4f}"
)

In [ ]:
print("WOE Mappings for Color_bin:")
woe_mappings = fastwoe_encoder.mappings_["Color_bin"]
display(woe_mappings)

print("\nKey WOE Values:")
print(
    f"Red (category 0): WOE = {woe_mappings.loc[0, 'woe']:.4f}, SE = {woe_mappings.loc[0, 'woe_se']:.4f}"
)
print(
    f"Blue (category 1): WOE = {woe_mappings.loc[1, 'woe']:.4f}, SE = {woe_mappings.loc[1, 'woe_se']:.4f}"
)

### Manual Verification of WOE Calculations:

Let's verify these WOE values manually:


In [ ]:
# Calculate log odds for each group
red_log_odds = np.log(30 / 10)  # 30 bad, 10 good for Red
blue_log_odds = np.log(10 / 20)  # 10 bad, 20 good for Blue
prior_log_odds = logit(40 / 70)  # 40 bad out of 70 total

print("Manual Calculation Verification:")
print(f"Red log odds: {red_log_odds:.4f}")
print(f"Blue log odds: {blue_log_odds:.4f}")
print(f"Prior log odds: {prior_log_odds:.4f}")

print("\nManual WOE calculations:")
red_woe = red_log_odds - prior_log_odds
blue_woe = blue_log_odds - prior_log_odds
print(f"Red WOE: {red_log_odds:.4f} - {prior_log_odds:.4f} = {red_woe:.4f}")
print(f"Blue WOE: {blue_log_odds:.4f} - {prior_log_odds:.4f} = {blue_woe:.4f}")

print("\nComparison with FastWoe:")
print(f"Red: Manual={red_woe:.4f}, FastWoe={woe_mappings.loc[0, 'woe']:.4f}")
print(f"Blue: Manual={blue_woe:.4f}, FastWoe={woe_mappings.loc[1, 'woe']:.4f}")


## 4. Standard Error Verification

Now let's verify that the standard errors from the contingency table match those from logistic regression:


In [ ]:
# Example data
red_bad, red_good = 30, 10
blue_bad, blue_good = 10, 20

red_se = (1 / red_bad + 1 / red_good) ** 0.5
blue_se = (1 / blue_bad + 1 / blue_good) ** 0.5
diff_se = (red_se**2 + blue_se**2) ** 0.5

logistic_0 = LogisticRegression()
logistic_0.fit(x, y)
logistic_0_se = logistic_0.summary()["standard_errors"][0]

logistic_1 = LogisticRegression()
logistic_1.fit(1 - x, y)
logistic_1_se = logistic_1.summary()["standard_errors"][0]

woe_se_red = red_se
woe_se_blue = blue_se

se_table = Table(
    title="WOE & Logistic Regression Standard Error Calculation",
    show_lines=True,
    min_width=60,
)
se_table.add_column("Group", style="bold cyan", justify="center")
se_table.add_column("Bad", justify="right")
se_table.add_column("Good", justify="right")
se_table.add_column("SE (Table)", justify="right")
se_table.add_column("SE (WOE)", justify="right")
se_table.add_column("SE (Logistic)", justify="right")

se_table.add_row(
    "Red",
    str(red_bad),
    str(red_good),
    f"{red_se:.4f}",
    f"{woe_se_red:.4f}",
    f"{logistic_0_se:.4f}",
)
se_table.add_row(
    "Blue",
    str(blue_bad),
    str(blue_good),
    f"{blue_se:.4f}",
    f"{woe_se_blue:.4f}",
    f"{logistic_1_se:.4f}",
)
se_table.add_row(
    "Difference",
    "-",
    "-",
    f"{diff_se:.4f}",
    f"{np.sqrt(red_se**2 + blue_se**2):.4f}",
    f"{np.sqrt(logistic_0_se**2 + logistic_1_se**2):.4f}",
)

# Center the table inside the panel with padding
panel = Panel(
    se_table,
    title="WOE Standard Error Verification",
    subtitle="All standard errors should match.",
    expand=False,
    padding=(1, 8),  # Top/bottom, left/right padding
)

console.print(panel)

## 5. WOE-Based Logistic Regression

Now let's fit a logistic regression using WOE-transformed features and compare the results:


In [ ]:
# Transform features using WOE
X_woe = fastwoe_encoder.transform(x)
print("WOE-transformed data (first 10 rows):")
print(pd.DataFrame(X_woe, columns=["Color_bin"]).sample(10))

# # Fit logistic regression on WOE-transformed data
logistic_woe = LogisticRegression(use_bias=True, max_iter=10)
logistic_woe.fit(X_woe, y)
print("\nWOE-based Logistic Regression:")
logistic_woe.display_summary(style="cyan1")

In [ ]:
woe_coef = logistic_woe.summary()["betas"][1]
woe_intercept = logistic_woe.summary()["betas"][0]
prior_log_odds = prior_log_odds  # already computed

# Calculate match for intercept
intercept_match = abs(woe_intercept - prior_log_odds) < 0.01

# Make the table
check_table = Table(
    title="WOE vs Logistic Regression Verification", show_lines=True, min_width=60
)
check_table.add_column("Test", style="bold cyan", justify="left")
check_table.add_column("Value", justify="right")
check_table.add_column("Expected", justify="right")
check_table.add_column("Match?", justify="center")

check_table.add_row(
    "WOE Coefficient",
    f"{woe_coef:.4f}",
    "≈ 1.0 (WOE contains log odds)",
    "v" if abs(woe_coef - 1.0) < 0.01 else "x",
)
check_table.add_row(
    "WOE Intercept",
    f"{woe_intercept:.4f}",
    f"Prior log odds: {prior_log_odds:.4f}",
    "v" if intercept_match else "x",
)

panel = Panel(
    check_table,
    title="Verification of WOE Coefficient & Intercept",
    subtitle="Are WOE/logistic values as expected?",
    expand=False,
    padding=(1, 8),
)

console.print(panel)

## Summary and Conclusions

This notebook has demonstrated several key relationships between Weight of Evidence (WOE) and standard errors:

### Main Findings:

1. **WOE is a centered version of log odds**: WOE = log odds - prior log odds
2. **Standard errors are identical**: SE(WOE) = SE(log odds) because subtracting a constant doesn't change variance
3. **Contingency table formulas work**: SE = √(1/n_bad + 1/n_good) matches logistic regression results
4. **WOE regression has special properties**: coefficient ≈ 1.0, intercept ≈ prior log odds

### Practical Implications:

- WOE transformations preserve all statistical properties of log odds
- Standard error calculations from contingency tables are valid for WOE
- WOE-based models are mathematically equivalent to log odds models
- The centering property of WOE doesn't affect inference or confidence intervals

## 6. Simulation: Variance Equality of Log Odds and WOE

This simulation demonstrates that **subtracting a constant (prior log odds) does not change variance**, which is why WOE and log odds have identical standard errors.

### Mathematical Property:
For any random variable X and constant c: Var(X - c) = Var(X)

In [ ]:
# Simple demonstration: Var(X - c) = Var(X)
np.random.seed(0)
x = np.random.randn(1000)
c = 5.0  # constant

print("Variance Property Demonstration:")
print(f"Var(X): {x.var():.6f}")
print(f"Var(X - c): {(x - c).var():.6f}")
print(f"Equal: {np.isclose(x.var(), (x - c).var())}")
print("\nThis is why WOE and log odds have identical standard errors!")

In [ ]:
# Empirical simulation with binary data
np.random.seed(42)

# True probabilities from our example
p1 = 30 / (30 + 20)  # Red group event rate
prior_p = (30 + 10) / (30 + 10 + 20 + 40)  # Overall event rate

# Sample sizes
n_total = 50  # Group total size
n_sim = 10_000  # Number of simulations

# Store results
log_odds_samples = []
woe_samples = []

for _ in range(n_sim):
    # Simulate binary outcomes for Red group
    y = np.random.binomial(1, p1, size=n_total)
    n_bad = y.sum()  # Number of bad outcomes
    n_good = n_total - n_bad  # Number of good outcomes

    # Compute log odds and WOE (avoid division by zero)
    if n_bad > 0 and n_good > 0:
        log_odds = np.log(n_bad / n_good)
        log_odds_prior = np.log(prior_p / (1 - prior_p))
        woe = log_odds - log_odds_prior

        log_odds_samples.append(log_odds)
        woe_samples.append(woe)

# Convert to arrays
log_odds_samples = np.array(log_odds_samples)
woe_samples = np.array(woe_samples)

# Compare empirical variances
variance_log_odds = np.var(log_odds_samples, ddof=1)
variance_woe = np.var(woe_samples, ddof=1)

# Add rich table
table = Table(title="Variance Comparison", show_lines=True, min_width=60)
table.add_column("Test", style="bold cyan", justify="left")
table.add_column("Value", justify="right")

table.add_row("Variance of log odds", f"{variance_log_odds:.4f}")
table.add_row("Variance of WOE", f"{variance_woe:.4f}")

console.print(table)